In [2]:
# Package Imports
import pickle
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [3]:
#Loading the Model
model = pickle.load(open('light_model.sav', 'rb'))

/home/greats/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.23.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [17]:
# Read in data
app_train = pd.read_csv('fapplication_train.csv').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000000, :]

In [18]:
# Set an index
app_train = app_train.set_index('SK_ID_CURR')

In [19]:
# Delete COLUMNS with very many NaNs, more than 40% of the observations in the column missing
app_data = app_train.dropna(thresh = 45000, axis = 1)

In [20]:
# Delete ROWS with very many NaNs, more than 40% of the observations in the row missing
app_data = app_data.dropna(thresh = 45, axis = 0)

In [21]:
# Delete ROWS with very many NaNs, more than 40% of the observations in the row missing
app_data = app_data.dropna(thresh = 45, axis = 0)
app_data

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,225000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,225000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,585000.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0


In [22]:
# Subset numerical data
numerics = ['int16','int32','int64','float16','float32','float64']
numerical_vars = list(app_data.select_dtypes(include=numerics).columns)
numerical_data = app_data[numerical_vars]
numerical_data.shape

(307511, 105)

In [23]:
# Fill in missing values
numerical_data = numerical_data.fillna(numerical_data.mean())

In [24]:
# Subset categorical data
cates = ['object']
cate_vars = list(app_data.select_dtypes(include=cates).columns)
categorical_data = app_data[cate_vars]
categorical_data

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
SK_ID_CURR,,,,,,,,,,,,,,,,
100002,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,"Stone, brick",No
100003,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,MONDAY,School,reg oper account,block of flats,Block,No
100004,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Government,NaN,NaN,NaN,NaN
100006,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,NaN,NaN,NaN,NaN
100007,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,Cash loans,M,N,N,Unaccompanied,Working,Secondary / secondary special,Separated,With parents,Sales staff,THURSDAY,Services,reg oper account,block of flats,"Stone, brick",No
456252,Cash loans,F,N,Y,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,NaN,MONDAY,XNA,reg oper account,block of flats,"Stone, brick",No
456253,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Separated,House / apartment,Managers,THURSDAY,School,reg oper account,block of flats,Panel,No


In [25]:
# Fill in missing values
categorical_data = categorical_data.fillna(method = 'ffill')
categorical_data

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
SK_ID_CURR,,,,,,,,,,,,,,,,
100002,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,"Stone, brick",No
100003,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,MONDAY,School,reg oper account,block of flats,Block,No
100004,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Government,reg oper account,block of flats,Block,No
100006,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,Block,No
100007,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion,reg oper account,block of flats,Block,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,Cash loans,M,N,N,Unaccompanied,Working,Secondary / secondary special,Separated,With parents,Sales staff,THURSDAY,Services,reg oper account,block of flats,"Stone, brick",No
456252,Cash loans,F,N,Y,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,Sales staff,MONDAY,XNA,reg oper account,block of flats,"Stone, brick",No
456253,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Separated,House / apartment,Managers,THURSDAY,School,reg oper account,block of flats,Panel,No


In [26]:
# Concat the data
clean_data = pd.concat([categorical_data, numerical_data], axis = 1)
clean_data

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,1.000000
100003,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000
100004,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000
100006,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,...,0,0,0,0,0.006402,0.007,0.034362,0.267395,0.265474,1.899974
100007,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,Cash loans,M,N,N,Unaccompanied,Working,Secondary / secondary special,Separated,With parents,Sales staff,...,0,0,0,0,0.006402,0.007,0.034362,0.267395,0.265474,1.899974
456252,Cash loans,F,N,Y,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,Sales staff,...,0,0,0,0,0.006402,0.007,0.034362,0.267395,0.265474,1.899974
456253,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Separated,House / apartment,Managers,...,0,0,0,0,1.000000,0.000,0.000000,1.000000,0.000000,1.000000


In [27]:
clean_data = clean_data[:1500]
clean_data

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,1.000000
100003,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000
100004,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000
100006,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,...,0,0,0,0,0.006402,0.007,0.034362,0.267395,0.265474,1.899974
100007,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101755,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Single / not married,House / apartment,Core staff,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,1.000000
101756,Cash loans,F,Y,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Low-skill Laborers,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,1.000000,1.000000
101760,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,High skill tech staff,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000


In [30]:
clean_data.to_csv(r'~/Documents/projects/dreatol/webapp/clean_data.csv',index=False)

In [26]:
# Instantiate label encoder
le = preprocessing.LabelEncoder()
categorical_data = categorical_data.apply(lambda col: le.fit_transform(col).astype(str))
categorical_data

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
SK_ID_CURR,,,,,,,,,,,,,,,,
100002,0,1,0,1,6,7,4,3,1,8,6,5,2,0,5,0
100003,0,0,0,0,1,4,1,1,1,3,1,39,2,0,0,0
100004,1,1,1,1,6,7,4,3,1,8,1,11,2,0,0,0
100006,0,0,0,1,6,7,4,0,1,8,6,5,2,0,0,0
100007,0,1,0,1,6,7,4,3,1,3,4,37,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,0,1,0,0,6,7,4,2,5,14,4,43,2,0,5,0
456252,0,0,0,1,6,3,4,5,1,14,1,57,2,0,5,0
456253,0,0,0,1,6,7,1,2,1,10,4,39,2,0,4,0


In [19]:
# Concat the data
clean_data = pd.concat([categorical_data, numerical_data], axis = 1)
clean_data

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,0,1,0,1,6,7,4,3,1,8,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,1.000000
100003,0,0,0,0,1,4,1,1,1,3,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000
100004,1,1,1,1,6,7,4,3,1,8,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000
100006,0,0,0,1,6,7,4,0,1,8,...,0,0,0,0,0.006402,0.007,0.034362,0.267395,0.265474,1.899974
100007,0,1,0,1,6,7,4,3,1,3,...,0,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,0,1,0,0,6,7,4,2,5,14,...,0,0,0,0,0.006402,0.007,0.034362,0.267395,0.265474,1.899974
456252,0,0,0,1,6,3,4,5,1,14,...,0,0,0,0,0.006402,0.007,0.034362,0.267395,0.265474,1.899974
456253,0,0,0,1,6,7,1,2,1,10,...,0,0,0,0,1.000000,0.000,0.000000,1.000000,0.000000,1.000000


In [15]:
# Preview Data
clean_data['TARGET'].sample(15)

SK_ID_CURR
450285    0
156198    0
440136    0
345406    0
142720    0
454766    0
205936    0
442075    0
189258    0
443664    0
187452    0
278722    0
213669    0
156753    0
156771    0
Name: TARGET, dtype: int64

In [16]:
# Prepare test data for individual predictions
test_data = clean_data.drop(['TARGET'], axis = 1)

In [17]:
# Define a risk assessment function
def risk_assessor(a):
    client_infor = test_data.loc[[a]].values   #Subset a specific client infor, *a* represent SK_ID_CURR
    prob = model.predict_proba(client_infor).tolist()[0]    #predict a client's probability of defaulting
    p = prob[1]
    if p > 0.67:
        print('Client with ID # {} has a high risk of defaulting the loan'.format(a))
    elif p > 0.33:
        print('Client with ID # {} has a moderate risk of defaulting the loan'.format(a))
    else:
        print('Client with ID # {} has a low risk of defaulting the loan'.format(a))

In [18]:
risk_assessor(261013)

NameError: name 'model' is not defined